In [1]:
import pandas as pd

import json
import glob
import os

In [2]:
base = "/amd.home/home/i11/zeitz/experiments/chpot/"
paths = glob.glob(base + "applications/*.json")
data = [json.load(open(path)) for path in paths]

In [3]:
queries = pd.DataFrame.from_records([{ 
    'exp': exp['experiment'], 
    'graph': run['args'][1],
    **algo } 
    for run in data for exp in run.get('experiments', []) for algo in exp['algo_runs'] if algo.get('algo') == 'CH Potentials Query'])

queries = queries.append(pd.DataFrame.from_records([{ 
    'exp': run['program'], 
    'graph': run['args'][1],
    **algo } 
    for run in data for algo in run.get('algo_runs', []) if algo.get('algo') in ['CH Potentials Query', 'CH Potentials TD Query']]))

In [9]:
queries['affected'] = queries['lower_bound'] != queries['result']
queries['increase'] = queries['result'] / queries['lower_bound']

In [19]:
table = queries.groupby(['graph', 'exp']).agg(
    running_time_ms=('running_time_ms', 'mean'), 
    num_pot_evals=('num_pot_evals', 'mean'), 
    num_relaxed_arcs=('num_relaxed_arcs', 'mean'),
    increase=('increase', 'mean'),
    affected=('affected', 'sum'),
    size=('affected', 'count'))
table['affected'] = table['affected'] * 100 / table['size']
table = table[table.columns[0:-1]]
table['num_relaxed_arcs'] = table['num_relaxed_arcs'] / 1000
table['num_pot_evals'] = table['num_pot_evals'] / 1000
#table = table.round(1)
print(table.to_latex())
table

\begin{tabular}{llrrrrr}
\toprule
                                         &            &  running\_time\_ms &  num\_pot\_evals &  num\_relaxed\_arcs &  increase &  affected \\
graph & exp &                  &                &                   &           &           \\
\midrule
/algoDaten/graphs/cleaned\_td\_road\_data/de/day/dido/ & chpot\_td &         4.829330 &     417.238270 &         33.088535 &  1.031095 &      99.5 \\
/algoDaten/graphs/cleaned\_td\_road\_data/ptv17-eur-car/day/di/ & chpot\_td &        98.458314 &    5143.787150 &        553.331675 &  1.039956 &      98.5 \\
/algoDaten/zeitz/roadgraphs/europe/ & fastest\_times\_10 &         1.043189 &      69.204050 &          2.740850 &  1.000004 &      16.5 \\
                                         & fastest\_times\_2 &         1.026094 &      68.923720 &          2.733065 &  1.000000 &      16.5 \\
                                         & random\_times\_10 &         1.095096 &      80.877945 &          3.049955 &  1.0000

running_time_ms  \
graph                                              exp                                 
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td                 4.829330   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td                98.458314   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10         1.043189   
                                                   fastest_times_2          1.026094   
                                                   random_times_10          1.095096   
                                                   random_times_2           1.041157   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live              96.484523   
                                                   no_highways            338.547844   
                                                   no_tunnels              39.397275   

                                                                     num_pot_evals  \
graph                                              exp                               
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td             417.238270   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td            5143.787150   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10      69.204050   
                                                   fastest_times_2       68.923720   
                                                   random_times_10       80.877945   
                                                   random_times_2        76.651035   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live          9310.909800   
                                                   no_highways        30383.142725   
                                                   no_tunnels          3874.148245   

                                                                     num_relaxed_arcs  \
graph                                              exp                                  
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td                 33.088535   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td                553.331675   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10          2.740850   
                                                   fastest_times_2           2.733065   
                                                   random_times_10           3.049955   
                                                   random_times_2            2.891315   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live              733.712270   
                                                   no_highways            2555.935385   
                                                   no_tunnels              290.581230   

                                                                     increase  \
graph                                              exp                          
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td          1.031095   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td          1.039956   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10  1.000004   
                                                   fastest_times_2   1.000000   
                                                   random_times_10   1.000028   
                                                   random_times_2    1.000003   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live        1.124989   
                                                   no_highways       6.244386   
                                                   no_tunnels        1.735734   

                                                                     affected  
graph                                              exp                         
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td              99.5  
/algoDaten/graphs/cleaned_td_road_data/pt